In [12]:
from unicodedata import category, normalize

letter = {'L'}
space = {'Z'}
letter_space = {'L', 'Z'}
dia = {'M'}
punc = {'P'}
letter_dia = {'L', 'M'}
udnorm = 'NFC'

test1a = '-:Κεκρ;?ότη-ται᾿,᾿'
test1b = 'Κεκρ;?ότηται᾿,᾿'
test1c = '-:Κε.:κρ;?ότη,.τα...ι᾿,᾿'
test2 = '''. A ...    Κεκ.,ρότη-ται;? . . .   ?κρη/πὶς ..ἀληθείας, ;ὦ παῖδες ὑμεῖς, ἡμῖν αὐτοῖς, 
ἁγίου νεὼ μεγάλου θεοῦ θεμέλιος γνώσεως ἀρραγής, προτροπὴ καλή, 
δι᾿ ὑπακοῆς εὐλόγου ζωῆς ἀιδίουὄρεξις, νοερῷ καταβληθεῖσα χωρίῳ.'''
test3 = '᾽κείν᾽'

In [13]:
def rsplitPunc(word, norm=udnorm, clean=False):
    '''This function splits off punctuation 
    from words on the RIGHT side of the word.
    
    returns (word, punc)
    '''
    w = normalize(norm, word)
    afterWord = len(w)
    for i in range(len(w) - 1, -1, -1):
        if category(w[i])[0] not in letter_dia:
            afterWord = i
        else:
            break
    if clean:
        return (''.join(c for c in w[0:afterWord] \
                          if category(c)[0] in letter_dia), w[afterWord:])
    else:
        return (w[0:afterWord], w[afterWord:])

In [14]:
print(rsplitPunc(test3, clean=True))

('κείν', '᾽')


In [ ]:
def lsplitPunc(word, norm=udnorm, clean=False):
    '''This function splits off punctuation 
    from words on the LEFT side of the word.
    
    returns (punc, word)
    '''
    w = normalize(norm, word)
    beforeWord = -1
    for i in range(len(w)):
        if category(w[i])[0] not in letter_dia:
            beforeWord = i
        else:
            beforeWord +=1
            break
    if clean:
        return (w[0:beforeWord], ''.join(c for c in w[beforeWord:] \
                                           if category(c)[0] in letter_dia))
    else:
        return (w[0:beforeWord], w[beforeWord:])


In [ ]:
print(lsplitPunc(test1a, clean=True))


In [ ]:
letter = {'L'}
space = {'Z'}
letter_space = {'L', 'Z'}
dia = {'M'}
punc = {'P'}
letter_dia = {'L', 'M'}
udnorm = 'NFC'

def splitPunc(words, norm=udnorm, clean=False,
              splitters=None, non_splitters=None):
    '''This function splits off punctuation 
    from words on both sides of the word. 
    It returns a tuple with tuples, containing
    the punctuation before, the word itself, 
    and punctuation after. It can be used for
    multiple words
    
    clean=False:
        if punctuation is within the word, the word
        will be split into two, except for characters
        defined in the non-splitters list.
    clean=True:
        punctuation within a word will be deleted, 
        except for characters defined in the splitters 
        list. In that case, the string will be split.
    
    
    splitters=['character', 'character', ...]
    non_splitters=['character', 'character', ...]
        
    
    returns ((pre, word, after), (pre, word, after), ...)
    '''
    if splitters is None: splitters = ()
    if non_splitters is None: non_splitters = ()
    w = normalize(norm, words)
    pP = 0
    for i in range(len(w)):
        if category(w[i])[0] in space and pP > 0:
            pP += 1
            preWord = w[0:pP].strip('\n')
            if preWord:
                rest = splitPunc(w[pP:], clean=clean, splitters=splitters, 
                         non_splitters=non_splitters) if pP < len(w) else ()
                return ((preWord, '', ''),) + rest
            else:
                continue
        elif category(w[i])[0] not in letter_dia:
            pP += 1  
        else:
            break
    preWord = w[0:pP].strip('\n') if pP else ''
    pW = pP
    for i in range(pP, len(w)):
        if w[i] in non_splitters:
            break
        elif category(w[i])[0] in letter_dia:
            pW += 1
        else:
            break
    word = w[pP:pW]
    pA = pW
    nsplit = False
    spaceBreak = False
    sLoc = None
    for i in range(pW, len(w)):
        if clean:
            if spaceBreak:
                if not category(w[i])[0] in letter_dia:
                    pA += 1
                    if category(w[i])[0] in space:
                        sLoc = pA
                else:
                    break
            elif category(w[i])[0] in space:
                pA += 1
                spaceBreak = True
                sLoc = pA
            elif w[i] in splitters:
                pA += 1
                break
            elif category(w[i])[0] in letter_dia:
                pW = i + 1
                pA = pW
                word += w[i]
            elif category(w[i])[0] not in letter_dia:
                pA += 1
        else:
            if spaceBreak:
                if not category(w[i])[0] in letter_dia:
                    pA += 1
                    if category(w[i])[0] in space:
                        sLoc = pA
                else:
                    break
            elif category(w[i])[0] in space:
                pA += 1
                spaceBreak = True
                sLoc = pA
            elif w[i] in non_splitters:
                nsplit = True
                continue
            elif category(w[i])[0] not in letter_dia:
                nsplit = False
                pA += 1
            elif category(w[i])[0] in letter_dia and nsplit == True:
                pW = i + 1
                pA = pW
                word += w[i]
            else:
                break
    if not sLoc:
        sLoc = pA
    afterWord = w[pW:sLoc].strip('\n')
    rest = splitPunc(w[sLoc:], clean=clean, splitters=splitters, 
                     non_splitters=non_splitters) if sLoc < len(w) else ()
    return ((preWord, word, afterWord),) + rest

In [ ]:
print(test2)
splitPunc(test2, clean=True, non_splitters=('-', '/',))

In [ ]:
def cleanWords(words, norm=udnorm, clean=False,
               splitters=None, non_splitters=None): 
    """cleanWords splits off any punctuation and 
    non-word characters from words in a string. 
    It can be used for cleaning single words,
    or to tokenize full sentences.
    
    clean=False:
        letter characters that have punctuation
        inbetween but no space, are split on punctuation
        exceptions can be defined in non_splitters
    
    clean=True:
        words with punctuation within (without whitespace) 
        are glued together without punctuation
        exceptions can be defined in splitters
    
    returns: ('string', 'string', ...)
    """
    if splitters is None: splitters = ()
    if non_splitters is None: non_splitters = ()
    w = normalize(norm, words)
    pP = 0
    for i in range(len(w)):
        if category(w[i])[0] not in letter_dia:
            pP += 1
        else:
            break
    pW = pP
    for i in range(pP, len(w)):
        if category(w[i])[0] in letter_dia:
            pW += 1
        else:
            break
    realWord = w[pP:pW]
    pA = pW
    nsplit = False
    for i in range(pW, len(w)):
        if clean:
            if category(w[i])[0] in space:
                break
            elif w[i] in splitters:
                break
            elif category(w[i])[0] not in letter_dia:
                pA += 1
            elif category(w[i])[0] in letter_dia:
                realWord += w[i]
                pA += 1
        else:
            if w[i] in non_splitters:
                nsplit = True
                continue
            elif category(w[i])[0] in letter_dia and nsplit == True:
                pW = i + 1
                pA = pW
                realWord += w[i]
            elif category(w[i])[0] not in letter_dia:
                nsplit = False
                pA += 1
            else:
                break
    res = (realWord,) + \
          (cleanWords(w[pA:], clean=clean, 
                      splitters=splitters, non_splitters=non_splitters) 
           if pA < len(w) else ())
    return res if not res == ('',) else ()


In [ ]:
print(test2)
cleanWords(test2, clean=True,
              splitters=('-'), non_splitters=('-', '/'))

In [ ]:
def tokenizer(sentence, norm=udnorm, punc=False, clean=False,
              splitter=None, non_splitter=None, func=None):
    """tokenize feeds a sentence string
    to splitWord, while concatenating the
    resulting strings into one tuple.
    
    clean=False:
        split on punctuation without whitespace
    clean=True:
        delete punctuation inside words
    clean=None
        
    returns: ('string', 'string', ...)
    """
    if func:
        func(sentence)
    else:
        if punc:
            if clean:
                return tuple(f'{pre}{word}{post}' \
                    for pre, word, post in splitPunc(sentence, norm=udnorm, clean=True,
                                                     splitter=splitter, non_splitter=non_splitter))
            else:
                return tuple(f'{pre}{word}{post}' \
                    for pre, word, post in splitPunc(sentence, norm=udnorm, clean=False,
                                                     splitter=splitter, non_splitter=non_splitter))
        else:
            if clean:
                return cleanWords(sentence, clean=True)
            else:
                return cleanWords(sentence, clean=False)


        

In [ ]:
error = {'punc': True, 'nop': False,}

def func(word, **kwargs):
    if kwargs['punc']:
        print(word.lower())

func('DEZE!', **error)

In [ ]:
dictio = {'publicationStmt': {'concat': True, 'delimit': ', ', 'end': '.'}}
for i in dictio:
    if 'end' in dictio[i]:
        print(i)
        print(dictio[i]['end'])

In [ ]:
import operator

attribs = {'a': 4, 'b': 1, 'c': 25, 'd': 1}
sorted_attribs = sorted(attribs.items(), key=operator.itemgetter(1))
print(sorted_attribs)

In [ ]:
from helpertools.unicodetricks import plainLow
from helpertools.data.greek_elisions import ELISIONS

ELISIONS_norm = {k.strip('᾽'): v for k, v in ELISIONS.items()}

def greekReplacements(word):
    if word in ELISION_norm:
        return ELISION_norm[word]
    plain_word = plainLow(word)
    # Deletion of movable-nu
    if plain_word.endswith(('εν', 'σιν', 'στιν')) and len(midWord_pl) >= 3:
        return word[:-1]
    # Handling final-sigma
    if plain_word.endswith('σ'):
        return word[:-1] + 'ς'
    # Handling various forms of ου
    if plain_word in ('ουχ', 'ουκ'):
        return word[:-1]
    # Handling ἐξ
    if plain_word == 'εξ':
        return word[:-1] + 'κ'
    
print(len(ELISIONS))
print(len(ELISIONS_norm))

In [ ]:
from lxml import etree

import xmlschema
import urllib.request
from os import path
from pprint import pprint

# url = 'http://www.stoa.org/epidoc/schema/latest/tei-epidoc.rng'
# scheme = urllib.request.urlretrieve(url, path.expanduser('~/github/pthu/tfbuilder/tfbuilder/temp/scheme.xsd'))
# xml_scheme = xmlschema.XMLSchema(url)
file = path.expanduser('~/github/pthu/sources/pt/tlg0555/tlg002/tlg0555.tlg002.opp-grc1.xml')

# pprint()
relaxng_doc = etree.parse(path.expanduser('~/github/pthu/tfbuilder/tfbuilder/temp/scheme.xsd'))
print(relaxng_doc)
relaxng = etree.RelaxNG(relaxng_doc)
print(relaxng)

doc = etree.parse(file)
print(doc)
relaxng.validate(doc)



In [ ]:
s = {'a|b'}
# print(s)
if 'a' in s:
    print('yes!')

In [ ]:
elem = ['dit is een testinstru-   ', 'ment om te kijken of het werkt.    ', 'en anders is het ge- ', 'woon een proef!']


data = ''.join([(line.strip() + ' ' if not line.strip().endswith('-') else line.strip()) for line in elem ])\
          .replace('<', '#!#<')\
          .replace('>', '>#!#')\
          .split('#!#')
print(data)

In [ ]:
s = 'LUDOVICUS OF DINDORFIUS.    '
if s.isupper():
    print(s.title())
    print(' '.join((c.title() if not c.lower() in {'of', 'the',} else c.lower()) for c in s.strip(' ').split()))


In [ ]:
st = 'name, sdfsdf '
# name = st[:st.find(',')]
name = st[st.find(',')+2:]
print(name)

In [ ]:
# import ray
# ray.init()

# @ray.remote
# def f(x):
#     return x * x

# futures = [f.remote(i) for i in range(4)]
# print(ray.get(futures))

In [ ]:
from multiprocessing import Pool

lis = [i for i in range(1000)]

def f(x):
    return print(x * x)

pool = Pool()
pool.map(f, lis)
pool.close()
pool.join()

In [ ]:
from os import path
from pprint import pprint
from cltk.corpus.greek.beta_to_unicode import Replacer
from cltk.corpus.greek.alphabet import filter_non_greek
# beta_to_uni.beta_code(word)

beta_to_uni = Replacer()

dictio = {}
endings = set()

with open(path.expanduser('~/github/pthu/sources/moveablenu.txt'), 'r+') as f:
    for line in f.readlines():
#         print(line)
        line = line.lower().replace('j', 's').replace('c', 'x').replace('\t', ' ')
#         print(line)
#         line = filter_non_greek(beta_to_uni.beta_code(line)).replace('σ ', 'ς ')
#         print(line)
        line = beta_to_uni.beta_code(line).replace('σ ', 'ς ')
#         print(line)
        nom, full = line.strip().split(' ', 1)
        if nom.endswith('ν'):
            dictio[nom] = full
    for i in dictio:
        endings.add(i[-3:])
#         print(line)
#     pprint(f.readlines())
pprint(dictio)
# pprint(endings)
# pprint(list(sorted(endings)))

In [ ]:
levels = 3
headers = [f'level {i}' for i in range(1, levels+1)]
headers.append('text')
print(headers)
print(headers[-8:-1])


In [ ]:
start = {'{',}
stop = {'}',}
head = '..{'
if set(head) & (start | stop):
    print('got it!')
    print(f'{head:<12} {start}')
    
s = 'dit is een tekst-'
x, y = s.rsplit(' ', 1)
print(x)
print(y)

In [ ]:
def func(a='1', b='2', **kwargs):
    for k, v in locals().items():
        print(f'{k} = {v}')
        
func(a='k', l='4', m='c', perfect_arg='super')

In [ ]:
pre_add = None
if pre_add:
    print("yes")

In [ ]:
word = 'word'
print(word[-3:])

In [ ]:
feat = {'a', 'b', 'c',}
dictio = {'m': 1, 'k': 5}
feat.update(dictio)
print(feat)


In [ ]:
pre = '<<'
post = '>>'
repl_word = 'dit is een test'

if len(repl_word.split(' ')) > 1:
    repl_word_split = list(enumerate(repl_word.split(' '), start=1))
    print(repl_word_split)
    pre_assigned = False
    result = []
    for n, w in repl_word_split:
        if not pre_assigned:
            result.append(tuple((pre, w, '')))
            pre_assigned = True
        elif n == len(repl_word_split):
            result.append(tuple(('', w, post)))
        else:
            result.append(tuple(('', w, '')))
    print(tuple(result))

In [ ]:
from collections import OrderedDict

token_out = OrderedDict([('pre', {'text': False, 'metadata': 'interpunction before word'}),
                      ('word', {'text': True, 'metadata': 'the original format of the word without interpunction'}),
                      ('post', {'text': False, 'metadata': 'interpunction after word'}),
                     ])
print(token_out)
for setting, value in enumerate(token_out):
    print(setting, value)
    
dictio = {}
dictio.update(token_out)
print(dictio)

In [ ]:
from pprint import pprint

langsettings = {
    'generic': {
        'slot_type': 'word',
        'intFeatures': set(),
        'nonIntFeatures': {'otype', 'oslots',},
        'struct_counter': dict(_sentence=1, _phrase=1),
        'generic': {},
    },
    'greek': {**langsettings['generic'], 
              'slot_type': 'letter',
              'generic': {'text': 'text'},
              'greekissues': 'no issues'}

}
pprint(langsettings)
pprint(langsettings['generic'])

In [ ]:
head_signs = {'start': {'{',},
              'stop' : {'}',},}
post = ' { '
print(post.split(''.join(head_signs['start'])))

In [ ]:
header = ['level1', 'level2', 'level3']
struct_counter = dict(_sentence=1, _phrase=1)

structs  = tuple(tuple('_book',) + tuple(header) + tuple(struct_counter))
print(structs)

In [ ]:
p = ''
p += 'abcd'
print(p)

In [ ]:
a = {'a',}
b = {'b',}
c = 'sadf'

if set(c) & (a | b):
    print(set(c))

In [ ]:
from pprint import pprint
a = {'a': {'.', ','},
     'b': f"de parameters zijn: {{{' '.join(a['a'])}}}",}
pprint(a)

In [ ]:
generic = {1:1}
greek = {**generic, 2:2}
latin = {3:3}
p = {'generic': generic, 
     'greek': greek,
     'latin': latin,}

print(p)

In [7]:
from collections import OrderedDict
from pprint import pprint

generic = {'token_out': OrderedDict([('pre', {'text': False, 'metadata': 'interpunction before word'}),
                                 ('orig', {'text': True, 'metadata': 'the original format of the word without interpunction'}),
                                 ('post', {'text': False, 'metadata': 'interpunction after word'}),
                                 ])}
# pprint(generic)

# for i, part in enumerate(generic['token_out'].items()):
#     print(i, part)
    

def token_features(token_out):
    featuresInd = []
    for i, (part, value) in enumerate(token_out.items()):
#         print(i)
#         print(part)
#         print(value)
        part = part
        if value['text'] == False:
            featuresInd.append((i, part))
    return tuple(featuresInd)

token_features(generic['token_out'])

pprint(generic)
s = {k for k in generic['token_out']} & {k for k in generic['token_out']}
print(s)

{'token_out': OrderedDict([('pre',
                            {'metadata': 'interpunction before word',
                             'text': False}),
                           ('orig',
                            {'metadata': 'the original format of the word '
                                         'without interpunction',
                             'text': True}),
                           ('post',
                            {'metadata': 'interpunction after word',
                             'text': False})])}
{'orig', 'pre', 'post'}


In [ ]:
def func(a=True, b=True, c=False, **kwargs):
    print(locals())
    kwargs.update(locals())
    print(kwargs)
    print(dictio['kwargs'])
    return kwargs

dictio = {'a': 1, 'b': 2}


for i, p in func(**dictio).items():
    print(i, p)



In [1]:
print('Σ')

Σ


In [14]:
m = False
if not type(m) == bool:
    print('yes')

s = {}

In [15]:
a = {'dit', 'dat', 'dut', 'dot', 'det'}
print(','.join(a))

dit,dat,dut,dot,det
